<a href="https://colab.research.google.com/github/AnastasiaZhivilo/Data-Science-Projects/blob/main/Crop_Boundary_Recognition/GeoTIFF_export_for_training_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initialise Google Drive and the Google Project cs88-2-capstone

In [3]:
#pip install rasterio

In [5]:
# Import necessary libraries
import ee
import geemap
import ee.batch
import os
from google.colab import drive
import rasterio
import matplotlib.pyplot as plt
import numpy as np

In [9]:
# 1. Mount Google Drive
drive.mount('/content/drive')
print("Google Drive mounted successfully.")

# 2. Authenticate and initialise Google Earth Engine

ee.Authenticate()
ee.Initialize(project='cs88-2-capstone')
print("Earth Engine authenticated and initialized.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully.
Earth Engine authenticated and initialized.


In [10]:

# -----------------------------------------------------------------

# 3. Create a geemap map object

# Set up Cloud masking function:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask).eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

# Define Place of Interest location, and date range
start_date = '2023-09-01'
end_date = '2024-09-15'


image = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate(start_date, end_date)
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
    .map(mask_s2_clouds)
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4','B3','B2'],
}


In [11]:

# 1. Define the central point
center_lon = 146.9561
center_lat = -34.7022
center_point = ee.Geometry.Point(center_lon, center_lat)

# 2. Define the size (radius) for the buffer in meters
buffer_radius_meters = 5000  # 15 kilometers

# 3. Create a geodesic buffer (circle) around the point
circle_geometry = center_point.buffer(buffer_radius_meters)

# 4. Get the Bounding Box (Rectangle) coordinates from the buffered geometry
# .bounds() returns an ee.Geometry.Rectangle
bounding_box_geom = circle_geometry.bounds()

bbox_coords = bounding_box_geom.getInfo()['coordinates'][0]

In [12]:
# Import the necessary component directly from ipyleaflet
try:
    from ipyleaflet import DrawControl
except ImportError:
    print("ipyleaflet DrawControl could not be imported. Please ensure geemap is installed.")
    # Stop execution if the critical component is missing
    raise

# Create a median composite of the filtered image collection
median_image = image.median().clip(bounding_box_geom)
print("Median image composite created and clipped to bounding box.")

# 1. Initialize the geemap interactive map
Map = geemap.Map(center=[center_lat, center_lon], zoom=12)

# 2. Add the clipped median image to the map
Map.add_ee_layer(median_image, visualization, 'Median Sentinel-2 Image')

# 3. Fit the map view to the bounding box
Map.zoom_to_bounds(bounding_box_geom.bounds().getInfo().get('coordinates')[0])
print("Map view adjusted to bounding box.")

# 4. Manually create the DrawControl object with TraitError-compliant arguments 🛠️
# The fix: Pass an empty dictionary {} to disable tools while satisfying the TraitError.
if DrawControl is not None:
    # 🌟 Configuration for the ONLY tool we want enabled: Polygon
    polygon_config = {
        'shapeOptions': {
            'fillColor': '#ff0000',
            'color': '#ff0000',
            'fillOpacity': 0.1,
            'weight': 2
        }
    }

    # Create the DrawControl, using {} to effectively disable tools
    draw_control = DrawControl(
        # The tool we want enabled
        polygon=polygon_config,

        # Tools we must disable by passing an EMPTY DICT {}
        polyline={}, # Disables polyline
        rectangle={}, # Disables rectangle
        circle={}, # Disables circle
        marker={}, # Disables marker

        # Control options
        edit=False,
        remove=False,
        position='topleft'
    )

    # Add the manually created control to the geemap object
    Map.add_control(draw_control)
    print("Drawing tools added to the map (Polygon only).")

    # 5. Define a function to execute when a drawing is completed
    def handle_draw(target, action, geo_json):
        """
        Callback function that runs when a user finishes drawing a feature.
        """
        # The geo_json argument contains the GeoJSON of the drawn feature
        if geo_json:
            geometry = geo_json.get('geometry')
            coordinates = geometry.get('coordinates')
            geometry_type = geometry.get('type')

            print(f"\n--- Drawn {geometry_type} Coordinates ---")
            if geometry_type == 'Polygon':
                # For a Polygon, the exterior ring coordinates are the first element of the list
                print(f"Exterior Ring Coordinates (GeoJSON format): {coordinates[0]}")
            else:
                print(f"Coordinates (GeoJSON format): {coordinates}")
            print("----------------------------------------")

            # Since the drawn feature is stored in geo_json, we don't need to manually clear a buffer.

    # 6. Attach the callback function to the control's 'on_draw' event
    draw_control.on_draw(handle_draw)

# 7. Display the map
print("Displaying interactive map. Use the Polygon tool (top-left) to draw polygons.")
Map

Median image composite created and clipped to bounding box.
Map view adjusted to bounding box.
Drawing tools added to the map (Polygon only).
Displaying interactive map. Use the Polygon tool (top-left) to draw polygons.


Map(center=[-34.7022, 146.9561], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…


--- Drawn Polygon Coordinates ---
Exterior Ring Coordinates (GeoJSON format): [[146.919866, -34.722905], [146.93377, -34.724915], [146.933341, -34.728724], [146.919136, -34.727067], [146.919866, -34.722905]]
----------------------------------------

--- Drawn Polygon Coordinates ---
Exterior Ring Coordinates (GeoJSON format): [[146.918192, -34.732534], [146.926904, -34.733592], [146.927848, -34.728231], [146.919136, -34.727137], [146.918192, -34.732534]]
----------------------------------------

--- Drawn Polygon Coordinates ---
Exterior Ring Coordinates (GeoJSON format): [[146.927161, -34.733627], [146.935229, -34.734685], [146.936259, -34.729042], [146.928191, -34.728266], [146.927161, -34.733627]]
----------------------------------------


In [13]:
folder_name = f"BBox_{center_lon:.4f}_{center_lat:.4f}".replace('.', '_').replace('-', '_')

In [14]:
folder_name

'BBox_146_9561__34_7022'

In [15]:


#folder_name = 'earthengine_exports'      # Folder in your Drive (e.g., /content/drive/MyDrive/earthengine_exports)
resolution_meters = 10                   # Spatial resolution of the GeoTIFF (e.g., 10m for Sentinel-2 data)

# --- GEOFFI EXPORT LOGIC ---

def export_polygon_to_geotiff(coords, name, folder, scale):
    """
    Takes GeoJSON coordinates, creates an Earth Engine geometry, rasterizes it,
    and exports the result as a GeoTIFF to Google Drive.
    The export region is limited to the bounds of the polygon itself.
    """
    try:
        # 1. Create the Earth Engine Polygon Geometry
        ee_polygon = ee.Geometry.Polygon(coords)
        print(f"\n--- Starting Export for {name} ---")
        print(f"Created ee.Geometry.Polygon with {len(coords)} vertices.")

        # CONVERT GEOMETRY TO FEATURE COLLECTION:
        ee_feature_collection = ee.FeatureCollection([ee.Feature(ee_polygon)])

        # 2. Rasterize the Geometry (Paint it onto an Image)
        empty_image = ee.Image.constant(0).rename('boundary')

        # Paint the feature collection onto the image, assigning a value (e.g., 1) to the polygon border.
        rasterized_image = empty_image.paint(
            featureCollection=ee_feature_collection,
            color=1,
            width=1 # Set width=1 for border-only
        ).reproject(crs='EPSG:4326', scale=scale)

        print("Geometry rasterized successfully.")

        # 3. Define Export Parameters and Start Task
        export_task = ee.batch.Export.image.toDrive(
            image=rasterized_image,
            description=name,
            folder=folder,
            fileNamePrefix=name,
            # REVERTED CHANGE: Use the polygon's bounding box for the export region
            region=ee_polygon.bounds(),
            scale=scale,
            fileFormat='GeoTIFF'
        )

        export_task.start()

        print(f"Task Name: {name} STARTED.")
        print(f"File will be saved to: Google Drive/{folder}/{name}.tif")

    except Exception as e:
        print(f"\nAn error occurred during export of {name}: {e}")
        print("Please ensure your coordinates are a list of lists and the polygon is closed.")


In [19]:

# --- BATCH EXECUTION ---

# Iterate over the list of polygons and start an export task for each one.
def batch_processing_polygons(polygons_to_export):
    print("--- Starting Batch GeoTIFF Export Tasks ---")
    for polygon in polygons_to_export:
        export_polygon_to_geotiff(
            coords=polygon['coordinates'],
            name=polygon['name'],
            folder=folder_name,
            scale=resolution_meters
        )
    print("\nAll export tasks have been initiated.")
    print("Check the Colab 'Tasks' tab or run 'ee.batch.data.getTaskList().getInfo()' to monitor progress.")




In [20]:
# This part is generated by AI. In order to generate paste the Coordinates that appear under the map
# when you create the polygons into a Gemini AI chat with following sentence:

# "Convert the following drawn polygon coordinates into a Python list named polygons_to_export,
#assigning sequential names starting with Polygon_1_Export:""

# For the coordinates copy them in the same format that appears under the map. ie:

# --- Drawn Polygon Coordinates ---
# Exterior Ring Coordinates (GeoJSON format): [[146.919866, -34.722905], [146.93377, -34.724915], [146.933341, -34.728724], [146.919136, -34.727067], [146.919866, -34.722905]]
# ----------------------------------------

# --- Drawn Polygon Coordinates ---
# Exterior Ring Coordinates (GeoJSON format): [[146.918192, -34.732534], [146.926904, -34.733592], [146.927848, -34.728231], [146.919136, -34.727137], [146.918192, -34.732534]]
# ----------------------------------------

#Then paste the output here.

# polygons_to_export = [
#     {
#         'name': 'Polygon_A_Export',
#         'coords': [
#             [146.889825, -34.768324], [146.896262, -34.769452], [146.897206, -34.763423], [146.891069, -34.762224], [146.889782, -34.767936], [146.889825, -34.768324]
#         ]
#     },
#     {
#         'name': 'Polygon_B_Export',
#         'coords': [
#             [146.89652, -34.769416], [146.903043, -34.770298], [146.903987, -34.76508], [146.897507, -34.763599], [146.89652, -34.769416]
#         ]
#     },
#     {
#         'name': 'Polygon_C_Export',
#         'coords': [
#             [146.903644, -34.770333], [146.909995, -34.771461], [146.910768, -34.766385], [146.904416, -34.764939], [146.903644, -34.770333]
#         ]
#     }
# ]

polygons_to_export = [
    {
        "name": "Polygon_1_Export",
        "coordinates": [
            [146.919866, -34.722905],
            [146.93377, -34.724915],
            [146.933341, -34.728724],
            [146.919136, -34.727067],
            [146.919866, -34.722905]
        ]
    },
    {
        "name": "Polygon_2_Export",
        "coordinates": [
            [146.918192, -34.732534],
            [146.926904, -34.733592],
            [146.927848, -34.728231],
            [146.919136, -34.727137],
            [146.918192, -34.732534]
        ]
    },
    {
        "name": "Polygon_3_Export",
        "coordinates": [
            [146.927161, -34.733627],
            [146.935229, -34.734685],
            [146.936259, -34.729042],
            [146.928191, -34.728266],
            [146.927161, -34.733627]
        ]
    }
]

In [21]:
batch_processing_polygons(polygons_to_export)

--- Starting Batch GeoTIFF Export Tasks ---

--- Starting Export for Polygon_1_Export ---
Created ee.Geometry.Polygon with 5 vertices.
Geometry rasterized successfully.
Task Name: Polygon_1_Export STARTED.
File will be saved to: Google Drive/BBox_146_9561__34_7022/Polygon_1_Export.tif

--- Starting Export for Polygon_2_Export ---
Created ee.Geometry.Polygon with 5 vertices.
Geometry rasterized successfully.
Task Name: Polygon_2_Export STARTED.
File will be saved to: Google Drive/BBox_146_9561__34_7022/Polygon_2_Export.tif

--- Starting Export for Polygon_3_Export ---
Created ee.Geometry.Polygon with 5 vertices.
Geometry rasterized successfully.
Task Name: Polygon_3_Export STARTED.
File will be saved to: Google Drive/BBox_146_9561__34_7022/Polygon_3_Export.tif

All export tasks have been initiated.
Check the Colab 'Tasks' tab or run 'ee.batch.data.getTaskList().getInfo()' to monitor progress.


In [108]:
# --- View Exported GeoTIFF in Colab ---
# This script requires the following libraries. If they are not installed, run:
# !pip install rasterio matplotlib


# Function to read and plot a single GeoTIFF
def plot_geotiff(file_name, folder_name, drive_path):
    """Reads a GeoTIFF and plots the boundary data."""

    geotiff_path = os.path.join(
        drive_path,
        'MyDrive',
        folder_name,
        f'{file_name}.tif'
    )

    print(f"Looking for file: {file_name}.tif")

    if not os.path.exists(geotiff_path):
        print(f"-> ERROR: File '{file_name}.tif' not found. Ensure the export task is COMPLETED.")
        return

    try:
        with rasterio.open(geotiff_path) as src:
            # Read the boundary band (which contains 1s for the border and 0s elsewhere)
            boundary_data = src.read(1)

            # Count how many non-zero pixels (borders) exist to ensure file isn't empty
            if np.sum(boundary_data) == 0:
                print(f"-> WARNING: File '{file_name}.tif' contains no boundary data (all zeros).")
                return

            # 5. Plot the data
            plt.figure(figsize=(8, 8))

            # Use 'interpolation=None' to show the sharp, 10m pixel boundaries
            # Use a colormap suitable for binary data (like 'Greys')
            plt.imshow(boundary_data, cmap='Greys', interpolation='none')

            plt.title(f'GeoTIFF Visualization: {file_name}.tif')
            plt.xlabel('Pixel Column')
            plt.ylabel('Pixel Row')

            # Since the image is just a border, we only need to show the value 1
            plt.colorbar(label='Boundary Value (1 = Border)', ticks=[0, 1])

            plt.show()
            print(f"Successfully displayed GeoTIFF for {file_name}.")

    except Exception as e:
        print(f"Error processing GeoTIFF file {file_name}: {e}")


In [109]:

# --- Batch Visualization ---


#folder_name = 'earthengine_exports'
# The base path where Google Drive is mounted
drive_mount_path = '/content/drive'


# Define the list of file names you want to view.
# UPDATE THIS LIST to match the 'name' field in your polygons_to_export list.
file_names_to_view = [
    'Polygon_A_Export',
    'Polygon_B_Export',
    'Polygon_C_Export'
]

for name in file_names_to_view:
    plot_geotiff(name, folder_name, drive_mount_path)

print("\n--- Batch Visualization Attempt Complete ---")


Looking for file: Polygon_A_Export.tif
-> ERROR: File 'Polygon_A_Export.tif' not found. Ensure the export task is COMPLETED.
Looking for file: Polygon_B_Export.tif
-> ERROR: File 'Polygon_B_Export.tif' not found. Ensure the export task is COMPLETED.
Looking for file: Polygon_C_Export.tif
-> ERROR: File 'Polygon_C_Export.tif' not found. Ensure the export task is COMPLETED.

--- Batch Visualization Attempt Complete ---
